<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/ExampleUseWandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

ConvAbuse is just the dictionary as a string lol

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install wandb

# Set up wandb

In [1]:
#import wandb
#wandb.login()#no idea how it automatically detects my acc

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eolian (capture_disagreement). Use `wandb login --relogin` to force relogin


True

In [4]:
import wandb
config = dict (
  architecture = "bert_arabic_english",
  epochs = 1,
  batch_size = 4
)

run = wandb.init(
  project="capture-disagreement",
  notes="test_run",
  #name = "test_run",
  config=config,
  save_code = True,
  tags = ["bert_arabic_english", "weighted_loss", "balanced_loss", "only_last_layer"],
  #magic = True
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
artifact_model = wandb.Artifact(name='bert_arabic_english_weighted_and_balanced_loss_only_last_layer', type='model')

# Load data

In [4]:
import pandas as pd

filepaths = ["/content/ArMIS_train.json", "/content/ConvAbuse_train.json", "/content/HS-Brexit_train.json", "/content/MD-Agreement_train.json"]
dfs = [pd.read_json(filepath) for filepath in filepaths]
dfs = [df.transpose() for df in dfs]
dfs = [df.astype({"hard_label": int}, errors='raise') for df in dfs]#other way aroud (do concat first) would make more sense

df_all = pd.concat(dfs)
#df_all["soft_label"] =  df_all["soft_label"].apply(lambda x: [x["0"], x["1"]])
#should probably do that, but then evaluate script has to be adapted

In [6]:
import pandas as pd
import os
artifact_dataset = run.use_artifact('capture_disagreement/capture-disagreement/dataset_combined_train:v0', type='dataset')
artifact_dir = artifact_dataset.download()
df_all = pd.read_csv(os.path.join(artifact_dir, "dataset_combined_train.csv"))

wandb:   1 of 1 files downloaded.  


# Dataset

In [7]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_all[["text", "hard_label", "soft_label"]])

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", do_lower_case=True)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/157 [00:00<?, ?B/s]

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=240)#This is the longest that roberta tokenizer would go, dunno

tokenized_datasets = dataset.map(tokenize_function, batched=False)

  0%|          | 0/10431 [00:00<?, ?ex/s]

In [10]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

In [11]:
tokenized_datasets.set_format("torch")#not sure if neccessary

In [12]:
from torch.utils.data import DataLoader

batch_size = 4
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=batch_size)

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", num_labels=2)

Some weights of the model checkpoint at lanwuwei/GigaBERT-v4-Arabic-and-English were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

In [12]:
wandb.watch(model, log_freq=100)

[]

In [13]:
for name, param in model.named_parameters():
      if name not in ["classifier.weight", "classifier.bias"]:
         param.requires_grad = False

In [14]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters())

In [15]:
from transformers import get_scheduler
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [16]:
import torch
from tqdm import tqdm_notebook
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
m = model.to(device)

In [17]:
import evaluate

#TODO something is off here, untrained model gets 0.6 im ce eval

def evaluate_f1_score_huggingface(model, eval_dataloader):
  total = 0
  f1 = 0
  f1_metric = evaluate.load("f1")
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    N = predictions.shape[0]
    total+=1#the metric already gives the average, so no + batch_size
    f1 += f1_metric.compute(predictions=predictions, references=hard_labels, average = 'micro')["f1"]#micro must be used, otherwise smth wrong is returned, TODO look into that (00 00 is undefined)

  return f1/total


#slightly different to the pytorch implementations, not sure why TODO
def evaluate_ce_score(model, eval_dataloader, epsilon = 1e-12):
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)                                      
    N = predictions.shape[0]
    total+=N#maybe should be 1 here as well
    ce += -torch.sum(soft_labels*torch.log(predictions+1e-9))
  return ce/total


def evaluate_bce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.BCELoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]["1"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    N = probabilities.shape[0]
    total +=N
    ce += criterion(probabilities[:,-1], soft_labels)
  return ce/total


def evaluate_ce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.CrossEntropyLoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)                          
    N = probabilities.shape[0]
    total+=N
    ce += criterion(probabilities, soft_labels)
  return ce/total



In [18]:
#TODO do this for each dataset independently

#weight = df_all.mean(axis=0)["hard_label"]
weight = df_all["hard_label"].mean()
weight = torch.tensor([weight, 1 - weight], device = device, dtype=torch.float32)#I think this is the right way around

In [ ]:

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):
      #there is also index_level_0 in the dataset, not sure if needed
      inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]
      #soft_labels = torch.cat([soft_labels["0"].unsqueeze(-1), soft_labels["1"].unsqueeze(-1)], dim=-1).to(device)# that can probably be done more intuitively
      outputs = model(input_ids=inputs, attention_mask = attention_mask)#NllLoss
      optimizer.zero_grad()
      loss = 0
      for j in range(outputs.logits.shape[0]):
        criterion = criterion = torch.nn.CrossEntropyLoss(weight = (torch.tensor([soft_labels["0"][j], soft_labels["1"][j]], device=device) + weight) / 2)#can probably be done more efficient
        loss += criterion(outputs.logits, hard_labels)
      loss = loss/batch_size
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      

      running_loss += loss.item()
      if i % 10 == 9:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 10))
        wandb.log({"loss": running_loss/10})
        running_loss = 0.0

torch.save(model.state_dict(), "model.pt")
artifact_model.add_file(local_path='model.pt')
run.log_artifact(artifact_model)

In [20]:
run.log_artifact(artifact_model)

In [21]:
r_ce = evaluate_ce_score(model, eval_dataloader)
wandb.log({"ce_held_out_score": r_ce})
r_ce

<ipython-input-17-e001fe9acb63>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/261 [00:00<?, ?it/s]

tensor(0.5418, device='cuda:0')

In [22]:
r_f1 = evaluate_f1_score_huggingface(model, eval_dataloader)
wandb.log({"f1_held_out_score": r_f1})
r_f1

<ipython-input-17-e001fe9acb63>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/261 [00:00<?, ?it/s]

0.7471264367816092

In [23]:
run.finish()

ce_held_out_score,▁
f1_held_out_score,▁
loss,▅▃▅▇█▁▂▄▆▄▆▃▆▇▃▆▆▆▅▅▅▄▆▅▇▂▅▂▄▆▃▅▂▆▃▄▅▄▃▂
ce_held_out_score,0.54181
f1_held_out_score,0.74713
loss,4.47496
